# Integrate tespy models into workflows

## Requirements for models in workflows

- models will be executed more than once
- specific results/processings/... will be retrieved frequently
- model crashes should be avoided or at least be handled in some way without
  leaving side effects (e.g. corrupting future models)
- there should be a way to exchange data (input and output) with the model
  in a structured way
- there might be the need for various tespy models doing similar things
  - various topologies
  - various working fluids
  - ...
- model inputs might change a lot (keep in mind: EVERY simulation has side effects!)

## (My) solution

disclaimer: This is not universal truth, this is what is working well for me.
Feedback/suggestions for improvements greatly appreciated!

- Structure
  - each model is wrapped in a class (make use of inheritance!)
  - the class has methods
    - to set up the Network and create a initial stable solution
    - to input component or connection parameters
    - to solve the model in design mode
    - to retrieve component or connection parameters
  - optionally, there are methods
    - to automatically generate cycle diagrams, QT diagrams
    - to solve the model in offdesign mode

- Parameter input and output
  - via dictionaries/json/yaml
  - keys are mapped to specific parameters in the model, e.g.
    "evaporator_pinch" mapped to the parameter **td_pinch** of the
    **component** **evaporator** 
  - there is a second structure in between: nested dictionaries to access
    components and connections
    - user specifies {"evaporator_pinch": 10}
    - parameter lookup -> "evaporator_pinch": ["Components", "evaporator", "td_pinch"]
    - nested dict -> {"Components": {"evaporatir": {"td_pinch": 10}}}
    - internally this dict is used to set values for model or retrieve results

- Solving methods
  - solve the model
  - check the status variable of the model
  - handle issues if status is not 0 or 1
  - handle what happens with converged simulation violating physical limits (status 1)
  - e.g. set a flat if the model has been solved successfully

  ## Template


In [ ]:
from tespy.tools.helpers import merge_dicts
from tespy.networks import Network


class ModelTemplate():

    def __init__(self, config) -> None:
        self.config = config
        self.parameter_lookup = self._parameter_lookup()
        self._create_network()

    def _create_network(self) -> None:
        self.nw = Network()
        self.nw.units.set_defaults(
            **self.config["units"]
        )

    def _parameter_lookup(self) -> dict:
        return {
            "evaporator_pinch": ["Components", "evaporator", "td_pinch"],
            "T_geo": ["Connections", "a1", "T"],
            "m_geo": ["Connections", "a1", "m"]
        }

    def _map_parameter(self, parameter: str) -> tuple:
        return self.parameter_lookup[parameter]

    def _map_to_input_dict(self, **kwargs) -> dict:
        input_dict = {}
        for param, value in kwargs.items():
            if param not in self.parameter_lookup:
                msg = (
                    f"The parameter {param} is not mapped to any input of the "
                    "model. The following parameters are available:\n"
                    f"{', '.join(self.parameter_lookup)}."
                )
                raise KeyError(msg)
            key = self._map_parameter(param)
            input_dict = merge_dicts(
                input_dict,
                {key[0]: {key[1]: {key[2]: value}}}
            )
        return input_dict

    def get_parameter(self, parameter: str) -> float:
        mapped = self._map_parameter(parameter)
        if mapped[0] == "Connections":
            return self.nw.get_conn(mapped[1]).get_attr(mapped[2]).val

        elif mapped[0] == "Components":
            return self.nw.get_comp(mapped[1]).get_attr(mapped[2]).val

    def set_parameters(self, **kwargs) -> None:
        input_dict = self._map_to_input_dict(**kwargs)
        if "Connections" in input_dict:
            for c, params in input_dict["Connections"].items():
                self.nw.get_conn(c).set_attr(**params)

        if "Components" in input_dict:
            for c, params in input_dict["Components"].items():
                self.nw.get_comp(c).set_attr(**params)

    def solve_model(self, **kwargs) -> None:
        self.set_parameters(**kwargs)

        self._solved = False
        self.nw.solve("design")

        if self.nw.status == 0:
            self._solved = True
        # is not required in this example, but could lead to handling some
        # stuff
        elif self.nw.status == 1:
            self._solved = False
        elif self.nw.status in [2, 3, 99]:
            # in this case model is very likely corrupted!!
            # fix it by running a presolve using the stable solution
            self._solved = False
            self.nw.solve("design", init_only=True, init_path=self._stable_solution)

    def solve_model_offdesign(self, design, **kwargs) -> None:
        self.set_parameters(**kwargs)

        self._solved = False
        self.nw.solve("offdesign", design_path=design)

        if self.nw.status == 0:
            self._solved = True
        # is not required in this example, but could lead to handling some
        # stuff
        elif self.nw.status == 1:
            self._solved = False
        elif self.nw.status in [2, 3, 99]:
            # in this case model is very likely corrupted!!
            # fix it by running a presolve using the stable solution
            self._solved = False
            self.nw.solve("offdesign", init_only=True, init_path=self._stable_solution, design_path=design)


from tespy.components import Turbine, MovingBoundaryHeatExchanger, CycleCloser, Pump, Source, Sink, Generator, PowerBus, PowerSink, Motor
from tespy.connections import Connection, PowerConnection


class ORCModel(ModelTemplate):

    def _create_network(self) -> None:

        super()._create_network()

        turbine = Turbine("turbine")
        recuperator = MovingBoundaryHeatExchanger("recuperator")
        condenser = MovingBoundaryHeatExchanger("condenser")
        pump = Pump("pump")
        preheater = MovingBoundaryHeatExchanger("preheater")
        evaporator = MovingBoundaryHeatExchanger("evaporator")
        cc = CycleCloser("cc")

        geo_source = Source("geo production")
        geo_sink = Sink("geo injection")

        air_source = Source("air source")
        air_sink = Sink("air sink")

        a1 = Connection(geo_source, "out1", evaporator, "in1", label="a1")
        a2 = Connection(evaporator, "out1", preheater, "in1", label="a2")
        a3 = Connection(preheater, "out1", geo_sink, "in1", label="a3")

        b1 = Connection(cc, "out1", turbine, "in1", label="b1")
        b2 = Connection(turbine, "out1", recuperator, "in1", label="b2")
        b3 = Connection(recuperator, "out1", condenser, "in1", label="b3")
        b4 = Connection(condenser, "out1", pump, "in1", label="b4")
        b5 = Connection(pump, "out1", recuperator, "in2", label="b5")
        b6 = Connection(recuperator, "out2", preheater, "in2", label="b6")
        b7 = Connection(preheater, "out2", evaporator, "in2", label="b7")
        b8 = Connection(evaporator, "out2", cc, "in1", label="b8")

        c1 = Connection(air_source, "out1", condenser, "in2", label="c1")
        c2 = Connection(condenser, "out2", air_sink, "in1", label="c2")

        self.nw.add_conns(a1, a2, a3, b1, b2, b3, b4, b5, b6, b7, b8, c1, c2)

        generator = Generator("generator")
        motor = Motor("motor")
        power_bus = PowerBus("bus", num_in=1, num_out=2)
        grid = PowerSink("grid")

        e1 = PowerConnection(turbine, "power", generator, "power_in", label="e1")
        e2 = PowerConnection(generator, "power_out", power_bus, "power_in1", label="e2")
        e3 = PowerConnection(power_bus, "power_out1", motor, "power_in", label="e3")
        e4 = PowerConnection(motor, "power_out", pump, "power", label="e4")
        e5 = PowerConnection(power_bus, "power_out2", grid, "power", label="e5")

        self.nw.add_conns(e1, e2, e3, e4, e5)

        generator.set_attr(eta=0.98)
        motor.set_attr(eta=0.98)

        a1.set_attr(fluid={"water": 1}, T=200, p=35, m=10)
        a2.set_attr(T=155)

        b1.set_attr(fluid={"Isopentane": 1}, x=1, T=150)

        b3.set_attr(td_dew=10, T_dew=30)
        b4.set_attr(td_bubble=5)
        b7.set_attr(td_bubble=5)

        c1.set_attr(fluid={"air": 1}, T=10, p=1)
        c2.set_attr(T=20)

        recuperator.set_attr(dp1=0, dp2=0)
        condenser.set_attr(dp1=0, dp2=0)
        preheater.set_attr(dp1=0, dp2=0)
        evaporator.set_attr(dp1=0, dp2=0)

        turbine.set_attr(eta_s=0.8)
        pump.set_attr(eta_s=0.7)

        self.nw.solve("design")

        b3.set_attr(T_dew=None)
        condenser.set_attr(td_pinch=5)

        a2.set_attr(T=None)
        evaporator.set_attr(td_pinch=10)

        condenser.set_attr(design=["td_pinch"], offdesign=["UA"])
        evaporator.set_attr(design=["td_pinch"], offdesign=["UA"])

        b3.set_attr(design=["td_dew"])
        b7.set_attr(design=["td_bubble"])
        preheater.set_attr(offdesign=["UA"])
        recuperator.set_attr(offdesign=["UA"])

        self.nw.solve("design")
        self._stable_solution = "stable_solution.json"
        self.nw.save(self._stable_solution)

## Most often: Issues with starting values

- Change of inputs might be problematic sometimes, especially in offdesign
- relax the changes:
  - make in-between simulations
  - make make more gradual changes

In [9]:
model = ORCModel({"units": {"temperature": "°C", "pressure": "bar"}})
model.nw.save("design.json")


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 1.48e+07   | 0 %        | 3.97e+00   | 4.36e+07   | 5.07e+06   | 0.00e+00   | 1.49e+06   
 2     | 6.82e+06   | 0 %        | 6.55e+02   | 1.89e+07   | 6.94e+06   | 0.00e+00   | 3.71e+06   
 3     | 1.07e+08   | 0 %        | 2.94e+02   | 1.02e+06   | 7.68e+05   | 0.00e+00   | 9.66e+07   
 4     | 2.50e+08   | 0 %        | 2.13e+01   | 3.72e+05   | 6.55e+04   | 0.00e+00   | 2.20e+08   
 5     | 3.48e+08   | 0 %        | 1.01e+00   | 1.02e+05   | 1.67e+04   | 0.00e+00   | 3.06e+08   
 6     | 1.08e+08   | 0 %        | 3.24e+00   | 1.18e+04   | 1.88e+04   | 0.00e+00   | 9.47e+07   
 7     | 4.09e+03   | 26 %       | 2.83e-02   | 7.15e+02   | 1.02e+02   | 0.00e+00   | 3.02e+03   
 8     | 9.22e-01   | 67 %       | 1.07e-04   | 2.25e+00   | 2.04e-01   | 0.00e+00   | 2.06e+00   
 9     | 

In [4]:
model.solve_model_offdesign("design.json", **{"T_geo": 165})

Simulation crashed due to an unexpected error:
unable to solve 1phase PY flash with Tmin=112.726, Tmax=423.15 due to error: HSU_P_flash_singlephase_Brent could not find a solution because Hmolar [-85769.5 J/mol] is below the minimum value of -25915.9195245 J/mol
Traceback (most recent call last):
  File "/home/francesco/gitprojects/tespy/src/tespy/networks/network.py", line 2489, in solve
    self.solve_loop(print_results=print_results)
    ~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/francesco/gitprojects/tespy/src/tespy/networks/network.py", line 2535, in solve_loop
    self.solve_control()
    ~~~~~~~~~~~~~~~~~~^^
  File "/home/francesco/gitprojects/tespy/src/tespy/networks/network.py", line 2925, in solve_control
    self.solve_equations()
    ~~~~~~~~~~~~~~~~~~~~^^
  File "/home/francesco/gitprojects/tespy/src/tespy/networks/network.py", line 2946, in solve_equations
    hlp.solve(obj, self.increment_filter)
    ~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/fra


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 1.54e+06   | 0 %        | 1.88e+02   | 1.36e+04   | 9.19e+04   | 0.00e+00   | 5.51e+05   
 2     | 1.56e+06   | 0 %        | 3.65e+02   | 4.58e+01   | 5.93e+05   | 0.00e+00   | 1.32e+06   
 3     | 5.40e+06   | 0 %        | 7.32e+01   | 3.66e+04   | 1.79e+06   | 0.00e+00   | 6.67e+04   


In [5]:
model.solve_model_offdesign("design.json", **{"T_geo": 180})
model.solve_model_offdesign("design.json", **{"T_geo": 165})


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 8.87e+05   | 0 %        | 1.08e+02   | 7.85e+03   | 5.29e+04   | 0.00e+00   | 3.17e+05   
 2     | 1.13e+05   | 10 %       | 8.31e+00   | 1.38e+02   | 1.37e+04   | 0.00e+00   | 4.13e+04   
 3     | 5.98e+03   | 24 %       | 2.18e+00   | 1.25e+02   | 2.95e+03   | 0.00e+00   | 7.26e+03   
 4     | 2.65e+02   | 39 %       | 9.31e-02   | 5.13e+00   | 1.35e+02   | 0.00e+00   | 3.07e+02   
 5     | 5.16e-01   | 69 %       | 1.48e-04   | 8.10e-03   | 2.38e-01   | 0.00e+00   | 4.93e-01   
 6     | 2.08e-06   | 100 %      | 7.19e-10   | 4.11e-08   | 1.02e-06   | 0.00e+00   | 2.20e-06   
 7     | 5.01e-08   | 100 %      | 2.03e-11   | 1.46e-09   | 2.68e-08   | 0.00e+00   | 5.97e-08   
Total iterations: 7, Calculation time: 0.19 s, Iterations per second: 36.44

 iter  | residual   | progress 

In [6]:
model.nw.set_attr(iterinfo=False)

T_geo_range = [200, 198, 165, 164, 175, 210, 200]
m_geo_range = [10, 7, 8, 10.5, 8, 9, 6]

for T_geo, m_geo in zip(T_geo_range, m_geo_range):
    model.solve_model_offdesign(
        "design.json",
        **{"T_geo": T_geo, "m_geo": m_geo}
    )

Simulation crashed due to an unexpected error:
unable to solve 1phase PY flash with Tmin=272.897, Tmax=515.707 due to error: HSU_P_flash_singlephase_Brent could not find a solution because Hmolar [-4625.5 J/mol] is below the minimum value of 44.1646561978 J/mol
Traceback (most recent call last):
  File "/home/francesco/gitprojects/tespy/src/tespy/networks/network.py", line 2489, in solve
    self.solve_loop(print_results=print_results)
    ~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/francesco/gitprojects/tespy/src/tespy/networks/network.py", line 2535, in solve_loop
    self.solve_control()
    ~~~~~~~~~~~~~~~~~~^^
  File "/home/francesco/gitprojects/tespy/src/tespy/networks/network.py", line 2925, in solve_control
    self.solve_equations()
    ~~~~~~~~~~~~~~~~~~~~^^
  File "/home/francesco/gitprojects/tespy/src/tespy/networks/network.py", line 2946, in solve_equations
    hlp.solve(obj, self.increment_filter)
    ~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/fran

In [7]:
import numpy as np

model.nw.set_attr(iterinfo=False)

model = ORCModel({"units": {"temperature": "°C", "pressure": "bar"}})
model.nw.save("design.json")
model.nw.set_attr(iterinfo=False)

T_geo_range = [200, 198, 165, 164, 175, 210, 200]
m_geo_range = [10, 7, 8, 10, 8, 9, 6]

for T_geo, m_geo in zip(T_geo_range, m_geo_range):

    T_geo_old = model.get_parameter("T_geo")
    T_geo_change = abs(T_geo - T_geo_old)
    m_geo_old = model.get_parameter("m_geo")
    m_geo_change = abs(m_geo - m_geo_old)

    T_geo_steps = int(T_geo_change // 3)
    m_geo_steps = int(m_geo_change // 0.2)

    num_steps = max(T_geo_steps, m_geo_steps)

    T_geo_steps = np.linspace(T_geo, T_geo_old, num_steps + 1, endpoint=False)[::-1]
    m_geo_steps = np.linspace(m_geo, m_geo_old, num_steps + 1, endpoint=False)[::-1]

    for T_geo, m_geo in zip(T_geo_steps, m_geo_steps):
        model.solve_model_offdesign(
            "design.json",
            **{"T_geo": T_geo, "m_geo": m_geo}
        )

    print(T_geo, m_geo, model.nw.get_conn("e5").E.val, model.nw.status == 0)


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 1.98e+06   | 0 %        | 3.40e+02   | 0.00e+00   | 6.56e+05   | 0.00e+00   | 2.03e+06   
 2     | 5.14e+06   | 0 %        | 5.14e-09   | 0.00e+00   | 4.42e+05   | 0.00e+00   | 1.14e+06   
 3     | 1.64e-01   | 75 %       | 0.00e+00   | 0.00e+00   | 1.09e-02   | 0.00e+00   | 8.01e-02   
 4     | 3.27e-09   | 100 %      | 4.63e-14   | 0.00e+00   | 1.05e-10   | 0.00e+00   | 3.09e-09   
 5     | 3.50e-10   | 100 %      | 0.00e+00   | 0.00e+00   | 3.52e-11   | 0.00e+00   | 2.55e-10   
Total iterations: 5, Calculation time: 0.01 s, Iterations per second: 623.74

 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 5.66e+00   | 58 %    

Invalid value for ttd_l: ttd_l = -19.9061011779238 below minimum value (0) at component evaporator.
Invalid value for ttd_min: ttd_min = -19.9061011779238 below minimum value (0) at component evaporator.
Invalid value for eff_hot: eff_hot = 1.7179427459087346 above maximum value (1) at component evaporator.
Invalid value for eff_max: eff_max = 1.7179427459087346 above maximum value (1) at component evaporator.
Invalid value for td_pinch: td_pinch = -24.246259088498164 below minimum value (0) at component evaporator.
The simulation converged but the calculated result nan watt / kelvin for the fixed input parameter UA is not equal to the originally specified value: 84569.23345484807. Usually, this can happen, when a method internally manipulates the associated equation during iteration in order to allow progress in situations, when the equation is otherwise not well defined for the currentvalues of the variables, e.g. in case a negative root would need to be evaluated.  Often, this can h

164.0 10.0 262913.72362598346 True
175.0 8.0 375592.0949419881 True
210.0 9.0 817046.5674598967 True
200.0 6.0 536171.604240352 True
